In [5]:
from transformers import AutoTokenizer, AutoModel, pipeline
import tensorflow_hub as hub

import pandas as pd
import numpy as np 
import pickle

## Read Data

In [6]:
ekg_denoised = pd.read_pickle('/home/sanjaycollege15/PredictingDiagnoses/Data/ekg_denoised_v2.pkl')
ekg_denoised = ekg_denoised.sample(frac=1)

In [7]:
ekg_denoised.head()

,ICD9_CODE,TEXT
62581,2,sinus bradycardia. left anterior fascicular bl...
12643,3,baseline artifact. probable underlying sinus r...
19435,0,sinus rhythm lateral st-t changes are nonspeci...
51284,2,dual chamber pacemaker with occasional prematu...
36386,1,sinus tachycardia with atrial bigeminy. non-sp...


## BioClinicalBERT

### Set up Model Pipeline

In [8]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", model_max_length=512)
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
pipe = pipeline('feature-extraction', model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Batching in order to avoid memory issues

In [9]:
features = pipe(['sinus rhythm. prolonged a-v conduction. left bundle-branch block. compared to the previous tracing of transmural inferior wall myocardial infarction was previously present if left bundle-branch block patterning is absent, although the patterning barely resembles left bundle-branch block. the current tracing has a wider qrs interval.',
                 'sinus rhythm with first degree atrio-ventricular conduction delay. compared to the previous tracing no diagnostic change. tracing 4'
                ], 
                padding='max_length'
               )



In [5]:
batched = np.array_split(ekg_denoised, 50)

In [ ]:
#for split_array in batched:
#    text_list = list(split_array['TEXT'])
#    features = pipe(text_list, padding='max_length')
#    feature_arrays.append(np.squeeze(features))

In [7]:
feature_arrays = []
text_list = list(batched[0]['TEXT'])
features = pipe(text_list, padding='max_length')
feature_arrays.append(np.squeeze(features))

In [11]:
with open('/home/sanjaycollege15/PredictingDiagnoses/Data/clinicalBert_1.pkl', 'wb') as f:
    pickle.dump(feature_arrays, f)

Output embeddings are more than 25GB! This is too big, need to figure out a way to incorporate it into the model so that only the relevant embeddings are pulled.